In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import matplotlib.pyplot as plt
import seaborn as sns
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/restaurant-revenue-prediction/train.csv.zip")


In [ ]:
train.sample(5)

In [ ]:
train = train.drop(["Id","Open Date"],axis=1)


In [ ]:
plt.figure(figsize=(10,10))
plt.bar(train.City,train.revenue)
plt.xlabel("City")
plt.ylabel("Revenue")
plt.tight_layout()

In [ ]:
plt.bar(train["City Group"],train.revenue)


In [ ]:
sns.countplot(train["City Group"])

In [ ]:
plt.bar(train.Type,train.revenue)
plt.xlabel("Type")
plt.ylabel("Revenue")

In [ ]:
numerical = train.select_dtypes(include=np.number)
categorical = train.select_dtypes(exclude=np.number)



In [ ]:
dummies = pd.get_dummies(categorical)


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
numerical_scaled = scaler.fit_transform(numerical)
numerical_scaled_df = pd.DataFrame(data=numerical_scaled,columns=numerical.columns,index=numerical.index)


In [ ]:
train_data = pd.concat([dummies,numerical_scaled_df],axis=1)


In [ ]:
X = train_data.drop("revenue",axis=1)
y = train_data.revenue

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
lin_reg = LinearRegression()
def plot_learning_curves(model,X,y):
    train_errors, val_errors = [],[]
    X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.2,random_state=42)
    for m in range(1,len(X_train)+1):
        model.fit(X_train[:m],y_train[:m])
        prediction_train = model.predict(X_train[:m])
        prediction_val = model.predict(X_val)
        mse_train = mean_squared_error(prediction_train,y_train[:m])
        train_errors.append(mse_train)
        mse_val = mean_squared_error(prediction_val,y_val)
        val_errors.append(mse_val)


    plt.figure(figsize=(10,10))
    plt.plot(train_errors,label="train",color="red")
    plt.plot(val_errors,label="Validation",color="blue")
    plt.xlabel("Training Set Size")
    plt.ylabel("Error")
    plt.axis([0, 110, 0, 10])                      
    plt.legend()
    

plot_learning_curves(lin_reg,X,y)

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
ridge = Ridge()
def plot_learning_curves(model,X,y):
    train_errors, val_errors = [],[]
    X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.2,random_state=42)
    for m in range(1,len(X_train)+1):
        model.fit(X_train[:m],y_train[:m])
        prediction_train = model.predict(X_train[:m])
        prediction_val = model.predict(X_val)
        mse_train = mean_squared_error(prediction_train,y_train[:m])
        train_errors.append(mse_train)
        mse_val = mean_squared_error(prediction_val,y_val)
        val_errors.append(mse_val)


    plt.figure(figsize=(10,10))
    plt.plot(train_errors,label="train",color="red")
    plt.plot(val_errors,label="Validation",color="blue")
    plt.xlabel("Training Set Size")
    plt.ylabel("Error")
    plt.axis([0, 110, 0, 4])                      
    plt.legend()
plot_learning_curves(ridge,X,y)

In [ ]:
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.2,random_state=42)


In [ ]:
from sklearn.linear_model import Lasso,ElasticNet,Ridge,SGDRegressor,LinearRegression
lasso = Lasso()
elastic = ElasticNet()
sgd_reg = SGDRegressor()
def predictor(X_train,y_train,X_val,y_val):
    sgd_reg.fit(X_train,y_train)
    predict_sgd = sgd_reg.predict(X_val)
    mse_sgd = mean_squared_error(predict_sgd,y_val)

    lasso.fit(X_train,y_train)
    predict_lasso = lasso.predict(X_val)
    mse_lasso = mean_squared_error(predict_lasso,y_val)

    lin_reg.fit(X_train,y_train)
    predict_lin = lin_reg.predict(X_val)
    mse_lin = mean_squared_error(predict_lin,y_val)

    ridge.fit(X_train,y_train)
    predict_ridge = ridge.predict(X_val)
    mse_ridge = mean_squared_error(predict_ridge,y_val)

    elastic.fit(X_train,y_train)
    predict_elastic = elastic.predict(X_val)
    mse_elastic = mean_squared_error(predict_elastic,y_val)

    return print(f"SGD {mse_sgd},Lasso {mse_lasso},Linear {mse_lin},Ridge {mse_ridge},Elastic {mse_elastic},min {min([mse_sgd,mse_lasso,mse_lin,mse_ridge,mse_elastic])}")

predictor(X_train,y_train,X_val,y_val)

In [ ]:
from sklearn.model_selection import cross_val_score

def cross_val_score_def(X,y,cv):
    lin_cross = cross_val_score(lin_reg,X,y,cv=cv,scoring="neg_mean_squared_error")
    sgd_cross = cross_val_score(sgd_reg,X,y,cv=cv,scoring="neg_mean_squared_error")
    lasso_cross = cross_val_score(lasso,X,y,cv=cv,scoring="neg_mean_squared_error")
    ridge_cross = cross_val_score(ridge,X,y,cv=cv,scoring="neg_mean_squared_error")
    elastic_cross = cross_val_score(elastic,X,y,scoring="neg_mean_squared_error")

    return print(f"SGD {sgd_cross.mean()},Lasso {lasso_cross.mean()},Linear {lin_cross.mean()},Ridge {ridge_cross.mean()},Elastic {elastic_cross.mean()}")


In [ ]:
cross_val_score_def(X_train,y_train,10)